In [6]:
from collections.abc import MutableMapping, Mapping
import re
import json


class DotDict(MutableMapping):
    def __init__(self, *args, **kwargs):
        self.__data = kwargs
        self.update(**kwargs)

    def __getitem__(self, key):
        try:
            return getattr(self, self.__convert_key(key))
        except AttributeError:
            raise KeyError(key)
    
    def __setitem__(self, key, value):
        print(f"__setitem__ called with {key} and {value}")
        setattr(self, self.__convert_key(key), value)

    def __delitem__(self, key):
        print(f"__delitem__ with '{key}'")
        del self.__data[self.__convert_key(key)]
        del self.__dict__[self.__convert_key(key)]
    
    def __setattr__(self, prop: str, value):
        if prop == "_DotDict__data":
            super().__setattr__(prop, value)
        else:
            if isinstance(value, (list, tuple)):
                prop_obj = self.create_prop_obj_for_list_recursively(value)
            elif isinstance(value, Mapping):
                prop_obj = DotDict(**value)
            else:
                prop_obj = value
            super().__setattr__(prop, prop_obj)
            self.__data[prop] = value

    def __iter__(self):
        iterator = iter(self.__dict__)
        next(iterator)  # first key is _DotDict__data, so should not be in view
        return iterator

    def __len__(self):
        return len(self.__data)

    def __str__(self):
        return json.dumps(self.to_dict(), indent=4)

    def to_dict(self):
        self.__data = {prop: self.call_to_dict_recursively(value) for prop, value in self.items()}
        return self.__data

    @classmethod
    def __convert_key(cls, key):
        key = re.sub(r'^\d+', '', str(key)).strip()
        return re.sub(r'\W+', '_', key)

    @classmethod
    def create_prop_obj_for_list_recursively(cls, value):
        # this method could be static, I just don't like static methods in Python
        if isinstance(value, (list, tuple)):
            prop_obj = []
            for v in value:
                prop_obj.append(cls.create_prop_obj_for_list_recursively(v))
        elif isinstance(value, Mapping):
            prop_obj = cls(**value)
        else:
            prop_obj = value
        return prop_obj

    @classmethod
    def call_to_dict_recursively(cls, value):
        if isinstance(value, cls):
            return value.to_dict()
        elif isinstance(value, (list, tuple)):
            result = []
            for v in value:
                result.append(cls.call_to_dict_recursively(v))
            return result
        else:
            return value


In [29]:
d = DotDict(k1="v1")
d.k2 = "new value 2"
d["key with space"] = {"nestedKey1": "nestedValue1"}
d.dotkey = DotDict(nestedDotKey="nestedDotValue")
print("d.key_with_space.nestedKey1 => ", d.key_with_space.nestedKey1)
print(d)

__setitem__ called with k1 and v1
__setitem__ called with key with space and {'nestedKey1': 'nestedValue1'}
__setitem__ called with nestedKey1 and nestedValue1
__setitem__ called with nestedDotKey and nestedDotValue
__setitem__ called with nestedDotKey and nestedDotValue
d.key_with_space.nestedKey1 =>  nestedValue1
{
    "k1": "v1",
    "k2": "new value 2",
    "key_with_space": {
        "nestedKey1": "nestedValue1"
    },
    "dotkey": {
        "nestedDotKey": "nestedDotValue"
    }
}


In [30]:
# How to create nested data and access list objects
d.dotkey.newNestedKey = ["nk1", {"nk2": ["nk3", "nk4", {"nk5": "nv5"}]}]
print(d.dotkey.newNestedKey[1].nk2[2].nk5)

__setitem__ called with nk2 and ['nk3', 'nk4', {'nk5': 'nv5'}]
__setitem__ called with nk5 and nv5
nv5


In [31]:
print(d)

{
    "k1": "v1",
    "k2": "new value 2",
    "key_with_space": {
        "nestedKey1": "nestedValue1"
    },
    "dotkey": {
        "nestedDotKey": "nestedDotValue",
        "newNestedKey": [
            "nk1",
            {
                "nk2": [
                    "nk3",
                    "nk4",
                    {
                        "nk5": "nv5"
                    }
                ]
            }
        ]
    }
}


In [8]:
d = DotDict(k1={"k2": "v2"})
for key, val in d.items():
    print(f"For key={key} value type is {type(val)}, val=\n{val}")

__setitem__ called with k1 and {'k2': 'v2'}
__setitem__ called with k2 and v2
For key=k1 value type is <class '__main__.DotDict'>, val=
{
    "k2": "v2"
}


In [18]:
d = DotDict()

d.k1 = {"k2": {"k3": "v3"}}
print(d._DotDict__data)
print(d)

__setitem__ called with k2 and {'k3': 'v3'}
__setitem__ called with k3 and v3
{'k1': {'k2': {'k3': 'v3'}}}
{
    "k1": {
        "k2": {
            "k3": "v3"
        }
    }
}


In [19]:
del d.k1.k2["k3"]

__delitem__ with 'k3'


In [20]:
print(d._DotDict__data)

{'k1': {'k2': {}}}
